<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [ ]:
from ultralytics import YOLO
import torch
from pathlib import Path
import json
from datetime import datetime
import psutil
from tqdm.auto import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class ModelCallback:
    def __init__(self, save_dir, epochs):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.best_map = 0
        self.training_history = []
        self.pbar = tqdm(total=epochs, desc="Training Progress")
        
    def on_train_epoch_end(self, trainer):
        metrics = trainer.metrics
        epoch = trainer.epoch
        current_map = metrics.get('metrics/mAP50-95(B)', 0)
        
        self.training_history.append({
            'epoch': epoch,
            'mAP': current_map,
            'metrics': metrics
        })
        
        with open(self.save_dir / 'training_history.json', 'w') as f:
            json.dump(self.training_history, f, indent=4)
        
        # Save best model
        if current_map > self.best_map:
            self.best_map = current_map
            best_path = self.save_dir / f'best_model_map_{current_map:.4f}.pt'
            torch.save(trainer.model.state_dict(), best_path)
            
        # Save last model
        last_path = self.save_dir / 'last_model.pt'
        torch.save(trainer.model.state_dict(), last_path)
        
        # Create a copy of last model with epoch info
        epoch_path = self.save_dir / f'last_model_epoch_{epoch}.pt'
        torch.save(trainer.model.state_dict(), epoch_path)
        
        self.pbar.update(1)
        self.pbar.set_postfix({'mAP': f'{current_map:.4f}'})

    def on_train_end(self, trainer):
        self.pbar.close()

In [ ]:
def check_system_usage():
    with tqdm(total=4, desc="System Check") as pbar:
        cpu = psutil.cpu_percent()
        pbar.update(1)
        ram = psutil.virtual_memory().percent
        pbar.update(1)
        gpu_allocated = gpu_reserved = 0
        if torch.cuda.is_available():
            gpu_allocated = torch.cuda.memory_allocated() / 1e9
            pbar.update(1)
            gpu_reserved = torch.cuda.memory_reserved() / 1e9
            pbar.update(1)
        return {
            'CPU': f'{cpu}%',
            'RAM': f'{ram}%',
            'GPU Allocated': f'{gpu_allocated:.2f} GB',
            'GPU Reserved': f'{gpu_reserved:.2f} GB'
        }

In [ ]:
def train_model(yaml_path, save_dir, epochs=100, imgsz=640, batch=16):
    print("Initializing model...")
    model = YOLO('yolov8n.pt')
    callback = ModelCallback(save_dir, epochs)
    
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=0 if torch.cuda.is_available() else 'cpu',
        patience=50,
        save=True,
        project=str(save_dir),
        name='train',
        exist_ok=True,
        callbacks=[callback]
    )
    
    return model, callback.best_map

In [ ]:
if __name__ == "__main__":
    yaml_path = '/content/drive/MyDrive/Photo person/data.yaml'
    save_dir = Path('/content/drive/MyDrive/pth_person_detect')
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = save_dir / f'run_{timestamp}'
    
    with tqdm(total=1, desc="Training Model") as pbar:
        model, best_map = train_model(yaml_path, run_dir, epochs=30)
        pbar.update(1)
    
    usage = check_system_usage()
    for metric, value in usage.items():
        print(f"{metric}: {value}")
    
    print(f"Best mAP: {best_map:.4f}")
    print(f"Models saved in: {run_dir}")